<a href="https://colab.research.google.com/github/sausterm/doggit/blob/master/scrape_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%%capture

!pip install selenium -q gwpy
!pip install Pillow -q gwpy
!apt-get update -q gwpy # to update ubuntu to correctly run apt install 
!apt install chromium-chromedriver -q gwpy
!cp /usr/lib/chromium-browser/chromedriver /usr/bin -q gwpy
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver') 
from selenium import webdriver
chrome_options = webdriver.ChromeOptions() 
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox') 
chrome_options.add_argument('--disable-dev-shm-usage') 
import numpy as np
from glob import glob
import os
import io
from PIL import Image
import hashlib
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from google.colab import drive

In [6]:
drive.mount('/content/drive')
os.chdir('drive/My Drive/Projects/DLND/dog_app')
os.listdir()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['scrape_images.ipynb',
 'haarcascades',
 'dogImages',
 'lfw',
 'test_imgs',
 'models',
 'ChromeDriver',
 'dog_app.ipynb',
 'model_dev.ipynb',
 'test_scraped_images']

In [7]:
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd.get('https://google.com')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [8]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:wd, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [9]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
      pass
        #print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        #print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
      pass
        #print(f"ERROR - Could not save {url} - {e}")

In [10]:
def search_and_download(search_term:str,driver_path:str,target_path,number_images=5):
    #target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_path):
        os.makedirs(target_path)

    """with webdriver.Chrome(executable_path=driver_path) as wd:"""
    res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
    print('urls fetched.')
    for elem in res:
        persist_image(target_path,elem)
    print('uploaded {} images'.format(search_term))

In [11]:
DRIVER_PATH= "./ChromeDriver/chromedriver"
search_term = 'Rock'
target_path = "./test_scraped_images"

search_and_download(
    search_term = search_term,
    driver_path = DRIVER_PATH,
    target_path = target_path
)

Found: 100 search results. Extracting links from 0:100
Found: 5 image links, done!
urls fetched.
uploaded Rock images


In [13]:
os.listdir()
dog_files_train = np.array(glob("dogImages/train/*/*"))
dog_files_valid = np.array(glob("dogImages/valid/*/*"))
dog_files_test = np.array(glob("dogImages/test/*/*"))

print("length of dog_files_train: ", len(dog_files_train), "\nlength of dog_files_valid: ", 
len(dog_files_valid), "\nlength of dog_files_test: ",  len(dog_files_test))
breeds = [] 

for i in range(len(dog_files_train)):
  breeds.append(str(dog_files_train[i]).split('/')[2][4:])
breeds = np.array(breeds)
breeds = np.unique(breeds)
breeds = list(breeds)

lst = [x for x in range(len(breeds))]
breeds = list(zip(lst,breeds))


length of dog_files_train:  19465 
length of dog_files_valid:  835 
length of dog_files_test:  836


In [25]:
%%capture
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


In [26]:

for i, breed in breeds[50:51]:
  
  n = str(round((0.001*(i+1)),3))[2:]
  while len(n)<3:
    n = n+"0"
  f = "{}.{}".format(n,str(breed))

  target_path = "./dogImages/train/{}".format(f)
  search_term = breed
  print("search term = {}".format(search_term))

  search_and_download(
    search_term = search_term,
    driver_path = DRIVER_PATH,
    target_path = target_path,
    number_images = 10
  )
  print(target_path)

search term = Chow_chow
Found: 100 search results. Extracting links from 0:100
Found: 10 image links, done!
urls fetched.
uploaded Chow_chow images
./dogImages/train/051.Chow_chow
